In [13]:
!pwd

/home/b/b382291/regridder/AQUA


In [1]:
import intake

In [29]:
# run mean OPA 
import os
# add directory where the utils.py file is located
import xarray as xr 
import importlib
import dask as dk
import numpy as np 

# for levante
os.chdir('/home/b/b382291/git/one_pass/src')
imported_module = importlib.import_module("opa")

os.chdir('/home/b/b382291/git/one_pass/src')
importlib.reload(imported_module)
from opa import *
from opa import opa # from module (file.py) import class name 

os.chdir('/home/b/b382291/regridder/AQUA')
from aqua import Reader
from aqua.reader import catalogue


# initalising the statistic 
stat1 = opa(statistic = "var", statFreq = "daily", outputFreq = "weekly", save = "false")

# list of possible statistics: mean, var, std, min, max, threshExceed, hist, percentile



Running the algorithm

In [34]:

for i in range(0, 24, 1): 
#
    ds1 = tasr.isel(time=slice(i,i+1)) # slice(start, stop, step), extract 'moving window' which is hourly data
    
    dm = stat1.compute(ds1)



In [ ]:
# from dask.distributed import LocalCluster, Client, progress
# processes = False
# nworker = 32
# ncpu = 32
# threads = ncpu // nworker
# print(f"Number of CPUs: {ncpu}, number of threads: {threads}, number of workers: {nworker}, processes: {processes}")
# client = Client(processes=processes,
#  threads_per_worker=threads,
#  n_workers=nworker,
#  memory_limit='64GB'
#  )

#catalogue();

# REGRIDDER 

os.chdir('/home/b/b382291/regridder/AQUA')

reader = Reader(model="IFS", exp="tco2559-ng5", source="ICMGG_atm2d", regrid="r020")
data = reader.retrieve(fix=False)
tasr = reader.regrid(data["2t"][:,:])

#tasr = reader.regrid(data["2t"][0:16,:])

In [37]:
# checking the output against np.mean 

ds2 = tasr.isel(time=slice(0,24)) # slice(start, stop, step), extract 'moving window' which is hourly data

axNum = ds2.get_axis_num('time')
npVar = np.var(ds2, axis = axNum, dtype = np.float64)
npVar = np.squeeze(npVar)

In [38]:
a= dm.stat - npVar
a.values

array([[[0.00112946, 0.00112946, 0.00112946, ..., 0.00112946,
         0.00112946, 0.00112946],
        [0.00105975, 0.00105975, 0.00105975, ..., 0.00105975,
         0.00105975, 0.00105975],
        [0.0042716 , 0.0042716 , 0.0042716 , ..., 0.0042716 ,
         0.0042716 , 0.0042716 ],
        ...,
        [0.00369817, 0.00369817, 0.00369817, ..., 0.00369817,
         0.00369817, 0.00369817],
        [0.0039031 , 0.0039031 , 0.0039031 , ..., 0.0039031 ,
         0.0039031 , 0.0039031 ],
        [0.00284868, 0.00284868, 0.00284868, ..., 0.00284868,
         0.00284868, 0.00284868]]])

In [16]:
loadPath = "/home/b/b382291/git/data/"
files = os.listdir(loadPath)
files = sorted(files)
savedData = xr.open_dataset(loadPath + files[0])


ds2 = tasr.isel(time=slice(0,24)) # slice(start, stop, step), extract 'moving window' which is hourly data

axNum = ds2.get_axis_num('time')
npMean = np.mean(ds2, axis = axNum, dtype = np.float64)
npMean = np.squeeze(npMean)

a = savedData.meanVar - npMean
a.values


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [17]:
ds2 = tasr.isel(time=slice(24,30)) # slice(start, stop, step), extract 'moving window' which is hourly data

axNum = ds2.get_axis_num('time')
npMean = np.mean(ds2, axis = axNum, dtype = np.float64)
npMean = np.squeeze(npMean)

a = meanStat.meanCum - npMean


NameError: name 'tasr' is not defined

In [ ]:
dm.stat

<xarray.DataArray 'stat' (time: 1, lat: 900, lon: 1800)>
dask.array<zeros_like, shape=(1, 900, 1800), dtype=float64, chunksize=(1, 900, 1800), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20T23:00:00
  * lon      (lon) float64 0.0 0.2 0.4 0.6 0.8 ... 359.0 359.2 359.4 359.6 359.8
  * lat      (lat) float64 -89.9 -89.7 -89.5 -89.3 -89.1 ... 89.3 89.5 89.7 89.9

In [32]:
stat1.varCum.compute()

array([[[0.61608502, 0.61608502, 0.61608502, ..., 0.61608502,
         0.61608502, 0.61608502],
        [0.56514207, 0.56514207, 0.56514207, ..., 0.56514207,
         0.56514207, 0.56514207],
        [2.3289892 , 2.3289892 , 2.3289892 , ..., 2.3289892 ,
         2.3289892 , 2.3289892 ],
        ...,
        [2.02783125, 2.02783125, 2.02783125, ..., 2.02783125,
         2.02783125, 2.02783125],
        [2.15079509, 2.15079509, 2.15079509, ..., 2.15079509,
         2.15079509, 2.15079509],
        [1.57241274, 1.57241274, 1.57241274, ..., 1.57241274,
         1.57241274, 1.57241274]]])